### En este notebook se debe de utilizar los archivos data_loader y processor para realizar la comparación y la estimación del número de entradas a la estación chamartín. 

In [1]:
import numpy as np
import seaborn as sb
import pandas as pd
import matplotlib.pyplot as plt
from processor2 import * #Importamos todas las clases del archivo processor

In [2]:
# #Verificamos con el df timeseries_o
# Period_Nn = pd.read_csv("/home/jonathan/tesis/AppDaniel/TFG_CODE2/app/data/trips_data/timeseries_o.csv", sep=',',index_col=0, parse_dates=True)
# Period_Nn = Period_Nn.loc['2020-06-21':'2020-07-31 23:00:00']
# columnas_o = [columna for columna in Period_Nn.columns if columna.startswith('2807905')]
# Period_Nn = Period_Nn[columnas_o]
# Period_Nn.tail(25)

# chamartin_o para el periodo de nueva normalidad: 21 Jun - 31 Jul 2020

In [3]:
Period_Nn = pd.read_csv("/home/jonathan/tesis/3er_avance/Trayectos_Periodo_Confinamiento2.csv", sep=',',index_col=0, parse_dates=True)
Period_Nn = Period_Nn.set_index('ds')
Period_Nn = Period_Nn.loc['2020-06-21':'2020-07-31 23:00:00']
columnas_o = [columna for columna in Period_Nn.columns if columna.startswith('2807905')]
Period_Nn = Period_Nn[columnas_o]
Period_Nn_2 = Period_Nn.copy()
# Realizamos la suma del distrito origen chamartín
Period_Nn['2807905'] = Period_Nn.loc[:, columnas_o].sum(axis=1)
colum_sum = Period_Nn.loc[:, '2807905']
index = Period_Nn.index
chamartin_o = pd.DataFrame({'2807905': colum_sum})
chamartin_o.index = index

In [4]:
chamartin_o.tail(5)

,2807905
ds,
2020-07-31 19:00:00,10303.203
2020-07-31 20:00:00,10160.696
2020-07-31 21:00:00,8482.825
2020-07-31 22:00:00,6300.449
2020-07-31 23:00:00,3954.628


# Estimación de las entradas a la estación Chamartín a partir de las predicciones del trayecto "Chamartín-Centro"

## Read - Archivos con las predicciones obtenidas

In [5]:
predictions1_0_5 = pd.read_csv("/home/jonathan/tesis/10mo_avance/Evaluación/predictions1_0_5.csv", sep=',',index_col=0, parse_dates=True)
predictions2_0_5 = pd.read_csv("/home/jonathan/tesis/10mo_avance/Evaluación/predictions2_0_5.csv", sep=',',index_col=0, parse_dates=True)
predictions1_6_11 = pd.read_csv("/home/jonathan/tesis/10mo_avance/Evaluación/predictions1_6_11.csv", sep=',',index_col=0, parse_dates=True)
predictions2_6_11 = pd.read_csv("/home/jonathan/tesis/10mo_avance/Evaluación/predictions2_6_11.csv", sep=',',index_col=0, parse_dates=True)
predictions1_12_17 = pd.read_csv("/home/jonathan/tesis/10mo_avance/Evaluación/predictions1_12_17.csv", sep=',',index_col=0, parse_dates=True)
predictions2_12_17 = pd.read_csv("/home/jonathan/tesis/10mo_avance/Evaluación/predictions2_12_17.csv", sep=',',index_col=0, parse_dates=True)
predictions1_18_23 = pd.read_csv("/home/jonathan/tesis/10mo_avance/Evaluación/predictions1_18_23.csv", sep=',',index_col=0, parse_dates=True)
predictions2_18_23 = pd.read_csv("/home/jonathan/tesis/10mo_avance/Evaluación/predictions2_18_23.csv", sep=',',index_col=0, parse_dates=True)

predictions1 = pd.concat([predictions1_0_5, predictions1_6_11, predictions1_12_17, predictions1_18_23])
predictions2 = pd.concat([predictions2_0_5, predictions2_6_11, predictions2_12_17, predictions2_18_23])

## Añadimos las predicciones en el df "Period_Nn_2" para obtener el df chamartin_op

In [6]:
valores_reemplazo_20jul = predictions1.loc['2020-07-20', 'yhat'].to_dict()
valores_reemplazo_27jul = predictions2.loc['2020-07-27', 'yhat'].to_dict()

# Reemplaza los registros específicos en la columna '2807905-2807901'
for fecha_hora, valor in valores_reemplazo_20jul.items():
    fecha = fecha_hora.date()
    hora = fecha_hora.time().strftime('%H:%M:%S')       
    Period_Nn_2.loc[f'{fecha} {hora}', '2807905-2807901'] = valor
    
for fecha_hora, valor in valores_reemplazo_27jul.items():
    fecha = fecha_hora.date()
    hora = fecha_hora.time().strftime('%H:%M:%S')       
    Period_Nn_2.loc[f'{fecha} {hora}', '2807905-2807901'] = valor

In [7]:
columnas_o = [columna for columna in Period_Nn_2.columns if columna.startswith('2807905')]
Period_Nn_2['2807905'] = Period_Nn_2.loc[:, columnas_o].sum(axis=1)
colum_sum = Period_Nn_2.loc[:, '2807905']
index = Period_Nn_2.index
chamartin_op = pd.DataFrame({'2807905': colum_sum})
chamartin_op.index = index

In [14]:
# Verificamos que se hayan reemplazado los registros por las prediccones
# print(chamartin_o.loc['2020-07-19 23:00:00':'2020-07-20 01:00:00'])
# print(chamartin_op.loc['2020-07-19 23:00:00':'2020-07-20 01:00:00'])
# print(chamartin_o.loc['2020-07-26 23:00:00':'2020-07-27 01:00:00'])
# print(chamartin_op.loc['2020-07-26 23:00:00':'2020-07-27 01:00:00'])